# Assignment 4

**Due to**: TBD

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Fact checking, Neural Languange Inference (**NLI**)

# Intro

This assignment is centred on a particular and emerging NLP task, formally known as **fact checking** (or fake checking). As AI techniques become more and more powerful, reaching amazing results, such as image and text generation, it is more than ever necessary to build tools able to distinguish what is real from what is fake.

Here we focus on a small portion of the whole fact checking problem, which aims to determine whether a given statement (fact) conveys a trustworthy information or not. 

More precisely, given a set of evidences and a fact to verify, we would like our model to correctly predict whether the fact is true or fake.

In particular, we will see:

*   Dataset preparation (analysis and pre-processing)
*   Problem formulation: multi-input binary classification
*   Defining an evaluation method
*   Simple sentence embedding
*   Neural building blocks
*   Neural architecture extension

# The FEVER dataset

First of all, we need to choose a dataset. In this assignment we will rely on the [FEVER dataset](https://fever.ai).

The dataset is about facts taken from Wikipedia documents that have to be verified. In particular, facts could face manual modifications in order to define fake information or to give different formulations of the same concept.

The dataset consists of 185,445 claims manually verified against the introductory sections of Wikipedia pages and classified as ```Supported```, ```Refuted``` or ```NotEnoughInfo```. For the first two classes, systems and annotators need to also return the combination of sentences forming the necessary evidence supporting or refuting the claim.

## Dataset structure

Relevant data is divided into two file types. Information concerning the fact to verify, its verdict and associated supporting/opposing statements are stored in **.jsonl** format. In particular, each JSON element is a python dictionary with the following relevant fields:

*    **ID**: ID associated to the fact to verify.

*    **Verifiable**: whether the fact has been verified or not: ```VERIFIABLE``` or ```NOT VERIFIABLE```.
    
*    **Label**: the final verdict on the fact to verify: ```SUPPORTS```, ```REFUTES``` or ```NOT ENOUGH INFO```.
    
*    **Claim**: the fact to verify.
    
*    **Evidence**: a nested list of document IDs along with the sentence ID that is associated to the fact to verify. In particular, each list element is a tuple of four elements: the first two are internal annotator IDs that can be safely ignored; the third term is the document ID (called URL) and the last one is the sentence number (ID) in the pointed document to consider.

**Some Examples**

---

**Verifiable**

{"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}

---

**Not Verifiable**

{"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}

---

## Some simplifications and pre-processing

We are only interested in verifiable facts. Thus, we can filter out all non-verifiable claims.

Additionally, the current dataset format does not contain all necessary information for our classification purposes. In particular, we need to download Wikipedia documents and replace reported evidence IDs with the corresponding text.

Don't worry about that! We are providing you the already pre-processed dataset so that you can concentrate on the classification pipeline (pre-processing, model definition, evaluation and training).

You can download the zip file containing all set splits (train, validation and test) of the FEVER dataset by clicking on this [link](https://drive.google.com/file/d/1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1/view?usp=sharing). Alternatively, run the below code cell to automatically download it on this notebook.

**Note**: each dataset split is in .csv format. Feel free to inspect the whole dataset!

In [1]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

# Classification dataset

At this point, you should have a reay-to-go dataset! Note that the dataset format changed as well! In particular, we split the evidence set associated to each claim, in order to build (claim, evidence) pairs. The classification label is propagated as well.

We'll motivate this decision in the next section!

Just for clarity, here's an example of the pre-processed dataset:

---

**Claim**: "Wentworth Miller is yet to make his screenwriting debut."

**Evidence**: "2	He made his screenwriting debut with the 2013 thriller film Stoker .	Stoker	Stoker (film)"

**Label**: Refutes

---

**Note**: The dataset requires some text cleaning as you may notice!


# Problem formulation

As mentioned at the beginning of the assignment, we are going to formulate the fact checking problem as a binary classification task.

In particular, each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

## Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: [batch_size, max_tokens]
*    Evidence: [batch_size, max_tokens]

Moreover, after the embedding layer, we'll have:

*    Claim: [batch_size, max_tokens, embedding_dim]
*    Evidence: [batch_size, max_tokens, embedding_dim]

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: [batch_size, dim]

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

*   Encode token sequences via a RNN and take the last state as the sentence embedding.

*   Encode token sequences via a RNN and average all the output states.

*   Encode token sequences via a simple MLP layer. In particular, if your input is a [batch_size, max_tokens, embedding_dim] tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a [batch_size, embedding_dim] 2D matrix.

*   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if we the sentence embedding of a single input has shape [batch_size, embedding_dim], then the classification input has shape:

*     **Concatenation**: [batch_size, 2 * embedding_dim]

*     **Sum**: [batch_size, embedding_dim]

*     **Mean**: [batch_size, embedding_dim]

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: [batch_size, 2 * embedding_dim + 1]

*     **Sum**: [batch_size, embedding_dim + 1]

*     **Mean**: [batch_size, embedding_dim + 1]



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

**Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

**Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

# Tips and Extras

## Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

Don't forget to try neural baselines first!

## Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

Don't forget that your feedback is very important! Your suggestions help us improving course material.

# FAQ

---

**Q: Can I do something text pre-processing?**

**A:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Q: I'm struggling with the implementation. Can you help me?**

**A:** Yes sure! Write us an email about your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Q: Can I try other encoding strategies or neural architectures?**

**A:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---

# PREPROCESSING

In [2]:
import pandas as pd
import re
from collections import Counter
from keras.preprocessing.text import Tokenizer
train = pd.read_csv('./dataset/train_pairs.csv')
test = pd.read_csv('./dataset/test_pairs.csv')
val = pd.read_csv('./dataset/val_pairs.csv')
train

,Unnamed: 0,Claim,Evidence,ID,Label
0,0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS
1,1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS
2,2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES
3,3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS
4,4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES
...,...,...,...,...,...
121735,121735,April was the month Anderson Silva was born.,0\tAnderson da Silva -LRB- -LSB- ˈɐ̃deʁsõ ˈsiw...,229440,SUPPORTS
121736,121736,Anderson Silva is an American Brazilian mixed ...,0\tAnderson da Silva -LRB- -LSB- ˈɐ̃deʁsõ ˈsiw...,229443,REFUTES
121737,121737,Anderson Silva is incapable of being a Brazili...,0\tAnderson da Silva -LRB- -LSB- ˈɐ̃deʁsõ ˈsiw...,229444,REFUTES
121738,121738,Anderson Silva was born on the month of April ...,0\tAnderson da Silva -LRB- -LSB- ˈɐ̃deʁsõ ˈsiw...,229445,SUPPORTS


In [3]:
ls = []

def combine_words(input,length):
    combined_inputs = []
    if len(input)>1:
        for i in range(len(input)-1):
            combined_inputs.append(input[i]+" "+last_word_of(input[i+1],length)) #add the last word of the right-neighbour (overlapping) sequence (before it has expanded), which is the next word in the original sentence
    return combined_inputs, length+1

def remove_duplicates(input, length):
    bool_broke=False #this means we didn't find any duplicates here
    for i in range(len(input) - length):
        if input[i]==input[i + length]: #found a duplicate piece of sentence!
            for j in range(0,length): #remove the overlapping sequences in reverse order
                del input[i + length - j]
            bool_broke = True
            break #break the for loop as the loop length does not matches the length of splitted_input anymore as we removed elements
    if bool_broke:
        return remove_duplicates(input, length) #if we found a duplicate, look for another duplicate of the same length
    return input

def last_word_of(input,length):
    splitted = input.split(" ")
    if len(splitted)==0:
        return input
    else:
        return splitted[length-1]

def prepare_evidence(df):
  ls = []
  index = 0
  for x in df['Evidence']:
    i = re.sub(r'^.*?\t', '', x)
    splitted_input = i.replace('-LRB-', '(').replace('-RRB- ', ')').replace('\t', ' ').replace('-LSB-', '').replace('-RSB-', '').split(' ')
    word_length = 1
    splitted_input,word_length = combine_words(splitted_input,word_length)

    intermediate_output = False

    while len(splitted_input)>1:
        splitted_input = remove_duplicates(splitted_input,word_length) #look whether two sequences of length n (with distance n apart) are equal. If so, remove the n overlapping sequences
        splitted_input, word_length = combine_words(splitted_input,word_length) #make even bigger sequences
        if intermediate_output:
            print(splitted_input)
            print(word_length)
    try:
      output = splitted_input[0]
    except:
      output = splitted_input
    index += 1
    ls.append(output)
  return(ls)

train_evidence = prepare_evidence(train)
test_evidence = prepare_evidence(test)
val_evidence = prepare_evidence(val)
print(train_evidence[0], test_evidence[0], val_evidence[0])

Hemsworth has also appeared in the science fiction action film Star Trek ( 2009 ), the thriller adventure A Perfect Getaway ( 2009 ), the horror comedy The Cabin in the Woods ( 2012 ), the dark-fantasy action film Snow White and the Huntsman ( 2012 ), the war film Red Dawn ( 2012 ), and the biographical sports drama film Rush ( 2013 ). Star Trek (film) A Perfect Getaway The Cabin in the Woods Snow White and the Huntsman Red Dawn (2012 film) Rush Rush (2013 film) Furthermore , anxiety has been linked with physical symptoms such as IBS and can heighten other mental health illnesses such as OCD and panic disorder . IBS Irritable bowel syndrome It is an all-volunteer force and comprises more than 80 % of the country 's active defence personnel . all-volunteer force Volunteer military


In [4]:
X_train = [(train.iloc[x]['Claim'], train_evidence[x]) for x in range(len(train_evidence))]
X_test = [(test.iloc[x]['Claim'], test_evidence[x]) for x in range(len(test_evidence))]
X_val = [(val.iloc[x]['Claim'], val_evidence[x]) for x in range(len(val_evidence))]
X_train[0]

('Chris Hemsworth appeared in A Perfect Getaway.',
 'Hemsworth has also appeared in the science fiction action film Star Trek ( 2009 ), the thriller adventure A Perfect Getaway ( 2009 ), the horror comedy The Cabin in the Woods ( 2012 ), the dark-fantasy action film Snow White and the Huntsman ( 2012 ), the war film Red Dawn ( 2012 ), and the biographical sports drama film Rush ( 2013 ). Star Trek (film) A Perfect Getaway The Cabin in the Woods Snow White and the Huntsman Red Dawn (2012 film) Rush Rush (2013 film)')

In [5]:
y_train = train['Label']
y_test = test['Label']
y_val = val['Label']

# CONVERSION

In [6]:
import gensim
import gensim.downloader as gloader
EMBEDDING_SIZE = 50
download_path = "glove-wiki-gigaword-{}".format(EMBEDDING_SIZE)
emb_model = gloader.load(download_path)

In [7]:
pos_dict = {}
def create_XY(df):
    X = [] # store input sequence
    pos = 0
    max_len = 0
    for sentence in df:
          X_sentence = []
          try:
            for word in sentence.split(' '):
              if len(sentence) > max_len:
                max_len = len(sentence)
              X_sentence.append(word)
          except:
              X_sentence.append('')
          X.append(X_sentence)
    num_words = len(set([word.lower() for sentence in X for word in sentence]))
    print("Total number of tagged sentences: {}".format(len(X)))
    print("Vocabulary size: {}".format(num_words))
    print('sample X: ', X[0], '\n')
    print("Length of first input sequence  : {}".format(len(X[0])))
    print("Maximum length of sentence : {}".format(max_len))
    return(X, num_words, max_len)
X_train_claim, num_words_train_claim, MAX_LEN_TRAIN_claim = create_XY(train['Claim'])
X_val_claim, num_words_val_claim, MAX_LEN_VAL_claim = create_XY(val['Claim'])
X_test_claim, num_words_test_claim, MAX_LEN_TEST_claim = create_XY(test['Claim'])

Total number of tagged sentences: 121740
Vocabulary size: 31186
sample X:  ['Chris', 'Hemsworth', 'appeared', 'in', 'A', 'Perfect', 'Getaway.'] 

Length of first input sequence  : 7
Maximum length of sentence : 614
Total number of tagged sentences: 7165
Vocabulary size: 6619
sample X:  ['The', 'Indian', 'Army', 'comprises', 'part', 'of', 'the', "country's", 'active', 'defense', 'personnel.'] 

Length of first input sequence  : 11
Maximum length of sentence : 200
Total number of tagged sentences: 7189
Vocabulary size: 7141
sample X:  ['Anxiety', 'has', 'been', 'linked', 'with', 'physical', 'symptoms.'] 

Length of first input sequence  : 7
Maximum length of sentence : 189


In [8]:
def create_XY(df):
    X = [] # store input sequence
    pos = 0
    max_len = 0
    for sentence in df:
      X_sentence = []
      try:
        for word in sentence.split(' '):
              if len(sentence) > max_len:
                max_len = len(sentence)
              X_sentence.append(word)
      except:
         X_sentence.append('')
      X.append(X_sentence)
    num_words = len(set([word.lower() for sentence in X for word in sentence]))
    print("Total number of tagged sentences: {}".format(len(X)))
    print("Vocabulary size: {}".format(num_words))
    print('sample X: ', X[0], '\n')
    print("Length of first input sequence  : {}".format(len(X[0])))
    print("Maximum length of sentence : {}".format(max_len))
    return(X, num_words, max_len)
X_train_evidence, num_words_train_evidence, MAX_LEN_TRAIN_evidence = create_XY(train_evidence)
X_val_evidence, num_words_val_evidence, MAX_LEN_VAL_evidence = create_XY(val_evidence)
X_test_evidence, num_words_test_evidence, MAX_LEN_TEST_evidence = create_XY(test_evidence)

Total number of tagged sentences: 121740
Vocabulary size: 36238
sample X:  ['Hemsworth', 'has', 'also', 'appeared', 'in', 'the', 'science', 'fiction', 'action', 'film', 'Star', 'Trek', '(', '2009', '),', 'the', 'thriller', 'adventure', 'A', 'Perfect', 'Getaway', '(', '2009', '),', 'the', 'horror', 'comedy', 'The', 'Cabin', 'in', 'the', 'Woods', '(', '2012', '),', 'the', 'dark-fantasy', 'action', 'film', 'Snow', 'White', 'and', 'the', 'Huntsman', '(', '2012', '),', 'the', 'war', 'film', 'Red', 'Dawn', '(', '2012', '),', 'and', 'the', 'biographical', 'sports', 'drama', 'film', 'Rush', '(', '2013', ').', 'Star', 'Trek', '(film)', 'A', 'Perfect', 'Getaway', 'The', 'Cabin', 'in', 'the', 'Woods', 'Snow', 'White', 'and', 'the', 'Huntsman', 'Red', 'Dawn', '(2012', 'film)', 'Rush', 'Rush', '(2013', 'film)'] 

Length of first input sequence  : 89
Maximum length of sentence : 1149
Total number of tagged sentences: 7165
Vocabulary size: 8586
sample X:  ['It', 'is', 'an', 'all-volunteer', 'force', 

In [9]:
words, tags = set([]), set([])
X_train = X_train_claim + X_train_evidence
 
for s in X_train:
    for w in s:
       words.add(w.lower())
 
word2index = {w: i + 1 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding

In [10]:
words_test, tags_test = set([]), set([])
X_test = X_test_claim + X_test_evidence

for s in X_test:
    for w in s:
       words_test.add(w.lower())
 
word2indexTest = {w: i + 2 for i, w in enumerate(list(words_test))}
word2indexTest['-PAD-'] = 0  # The special value used for padding

In [11]:
words_val, tags_val = set([]), set([])
X_val = X_val_claim + X_val_evidence

for s in X_val:
    for w in s:
       words_val.add(w.lower())
 
word2indexVal = {w: i + 2 for i, w in enumerate(list(words_val))}
word2indexVal['-PAD-'] = 0  # The special value used for padding

# I THINK THIS IS THE RIGHT INTERPRETATION OF THE ASSIGNMENT, BUT I AM NOT 100% SURE 

In [12]:
def merge_word2indexes(word2index, word2indexTest, word2indexVal): 
  for x in word2indexVal:
    if x not in word2index.keys():
      word2index[x] = len(word2index) 
  for x in word2indexTest:
    if x not in word2index.keys():
      word2index[x] = len(word2index) 
  return word2index
word2index = merge_word2indexes(word2index, word2indexTest, word2indexVal)

In [13]:
train_sentences_X_claim, test_sentences_X_claim, val_sentences_X_claim = [], [], []
train_sentences_X_evidence, test_sentences_X_evidence, val_sentences_X_evidence = [], [], []

oov = [] 

for s in X_train_claim:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    train_sentences_X_claim.append(s_int)

for s in X_train_evidence:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    train_sentences_X_evidence.append(s_int)

for s in X_test_claim:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    test_sentences_X_claim.append(s_int)

for s in X_test_evidence:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    test_sentences_X_evidence.append(s_int)

for s in X_val_claim:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    val_sentences_X_claim.append(s_int)

for s in X_val_evidence:
    s_int = []
    for w in s:
        s_int.append(word2index[w.lower()])
    val_sentences_X_evidence.append(s_int)
 
print(train_sentences_X_claim[0])
print(test_sentences_X_claim[0])
print(val_sentences_X_claim[0])
print(train_sentences_X_evidence[0])
print(test_sentences_X_evidence[0])
print(val_sentences_X_evidence[0])

[33884, 33619, 42014, 27725, 25382, 32936, 6458]
[29312, 4783, 11423, 32107, 51249, 3365, 52081]
[11427, 38464, 3426, 10143, 42414, 45336, 11427, 52067, 49297, 1798, 53797]
[33619, 4783, 374, 42014, 27725, 11427, 8448, 19688, 12029, 34984, 37809, 24413, 27410, 25422, 21833, 11427, 40155, 41857, 25382, 32936, 32615, 27410, 25422, 21833, 11427, 18678, 48158, 11427, 13627, 27725, 11427, 12236, 27410, 27808, 21833, 11427, 34293, 12029, 34984, 13142, 9313, 31307, 11427, 20000, 27410, 27808, 21833, 11427, 18453, 34984, 18283, 31540, 27410, 27808, 21833, 31307, 11427, 28488, 16972, 29617, 34984, 18776, 27410, 17341, 12470, 37809, 24413, 51842, 25382, 32936, 32615, 11427, 13627, 27725, 11427, 12236, 13142, 9313, 31307, 11427, 20000, 18283, 31540, 19376, 47515, 18776, 18776, 1549, 47515]
[22285, 18708, 29312, 4783, 11423, 32107, 51249, 3365, 35253, 735, 33199, 57015, 31307, 15019, 58079, 15247, 17119, 51410, 37940, 735, 33199, 56480, 31307, 25790, 8163, 31684, 57015, 55778, 1281, 3258]
[28703, 

In [14]:
from keras.preprocessing.sequence import pad_sequences

train_sentences_X_claim = pad_sequences(train_sentences_X_claim, maxlen=MAX_LEN_TRAIN_claim, padding='post')
test_sentences_X_claim = pad_sequences(test_sentences_X_claim, maxlen=MAX_LEN_TEST_claim, padding='post')
val_sentences_X_claim = pad_sequences(val_sentences_X_claim, maxlen=MAX_LEN_VAL_claim, padding='post')
train_sentences_X_evidence = pad_sequences(train_sentences_X_evidence, maxlen=MAX_LEN_TRAIN_evidence, padding='post')
test_sentences_X_evidence = pad_sequences(test_sentences_X_evidence, maxlen=MAX_LEN_TEST_evidence, padding='post')
val_sentences_X_evidence = pad_sequences(val_sentences_X_evidence, maxlen=MAX_LEN_VAL_evidence, padding='post')

In [15]:
import numpy as np
weights = np.zeros((len(word2index.keys()), EMBEDDING_SIZE))
word_list = list(word2index.keys())

oov = {}
for x in range(len(word_list)):
  try: 
    weights[x, :] = emb_model[word_list[x]]
  except:
    oov[word_list[x]] = x
    weights[x, :] = np.random.rand(EMBEDDING_SIZE)
    # TODO: controllare bounds per generare l'array random di pesi coerentemente
    # con l'embedding model di GloVe

# MODEL

In [16]:
import tensorflow as tf
from keras.layers import Add, Average, Dense, Input, GlobalMaxPooling1D, Dot, LSTM, Bidirectional, Dropout, Embedding, concatenate, Concatenate, GlobalAveragePooling1D
from keras.models import Model
from keras.optimizers import SGD
import tensorflow.keras as keras

In [17]:
VOCAB_SIZE = len(word2index)
BATCH_SIZE = 64
EMBEDDING_DIM = EMBEDDING_SIZE
UNITS = 256

In [18]:
claim_input = Input(shape=(MAX_LEN_TRAIN_claim), name='claim_input')
evidence_input = Input(shape=(MAX_LEN_TRAIN_evidence), name='evidence_input')

embedding_layer = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,  weights=[weights], mask_zero=True, trainable=False)

claim_outputs = embedding_layer(claim_input)
evidence_outputs = embedding_layer(evidence_input)

## Reduce the token sequence to a single representation (choose one)

### RNN taking last hidden state

In [19]:
def last_state_model():  
  claim_outputs1, claim_forward_h, _, claim_backward_h, _ = Bidirectional(LSTM(UNITS, return_sequences=True, return_state=True))(claim_outputs)
  claim_outputs1 = Concatenate()([claim_forward_h, claim_backward_h])

  evidence_outputs1, evidence_forward_h, _, evidence_backward_h, _ = Bidirectional(LSTM(UNITS, return_sequences=True, return_state=True))(evidence_outputs)
  evidence_outputs1 = Concatenate()([evidence_forward_h, evidence_backward_h])
  return claim_outputs1, evidence_outputs1

### RNN averaging all states

In [20]:
def avg_model():  
  claim_outputs2 = Bidirectional(LSTM(UNITS, return_sequences=True))(claim_outputs)
  evidence_outputs2 = Bidirectional(LSTM(UNITS, return_sequences=True))(evidence_outputs)

  claim_outputs2 = GlobalAveragePooling1D()(claim_outputs2)
  evidence_outputs2 = GlobalAveragePooling1D()(evidence_outputs2)
  return claim_outputs2, evidence_outputs2

### Using MLP layer

In [21]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [22]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.maxpool = GlobalMaxPooling1D()
        self.linear_1 = Linear(512)
        self.linear_2 = Linear(256)
        self.linear_3 = Linear(64)
        self.dropout = Dropout(0.1)

    def call(self, inputs):
        x = self.maxpool(inputs)
        x = self.linear_1(x)
        x = tf.nn.relu(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        x = self.dropout(x)
        return self.linear_3(x)

In [23]:
def mlp_model():  
  mlp = MLPBlock()
  claim_outputs3 = mlp(claim_outputs)
  evidence_outputs3 = mlp(evidence_outputs)
  return claim_outputs3, evidence_outputs3

### Using BoW

In [24]:
import keras.layers as L
import keras.backend as K

class NonZeroMean(L.Layer):
  def call(self, x): 
    """Calculate non-zero mean."""
    # count the number of nonzero features on embedding axis
    nonzero = K.any(K.not_equal(x, 0.0), axis=-1)
    n = K.sum(K.cast(nonzero, 'float32'), axis=-1, keepdims=True)
    # average on max_tokens axis
    x_mean = K.sum(x, axis=-2) / n
    return x_mean

  def compute_output_shape(self, input_shape):
    """Collapse summation axis."""
    return input_shape[:-2] + (input_shape[-1],)

In [25]:
def bow_model():
  claim_outputs4 = NonZeroMean()(claim_outputs)
  evidence_outputs4 = NonZeroMean()(evidence_outputs)
  return claim_outputs4, evidence_outputs4

## Final part of the model

In [26]:
def concat_model(claim_model, evidence_model):
  claim = Dropout(0.2)(claim_model.output)
  evidence = Dropout(0.2)(evidence_model.output)
  combined = concatenate([claim, evidence, sentence_similarity], name='combined_output')

  combined = Dense(64, activation='relu')(combined)
  combined = Dense(32, activation='relu')(combined)
  z = Dense(1, activation='sigmoid')(combined)
  return z

In [27]:
def sum_model(claim_model, evidence_model):
  claim = Dropout(0.2)(claim_model.output)
  evidence = Dropout(0.2)(evidence_model.output)
  combined = Add()([claim, evidence])

  combined = Dense(64, activation='relu')(combined)
  combined = Dense(32, activation='relu')(combined)
  z = Dense(1, activation='sigmoid')(combined)
  return z

In [28]:
def mean_model(claim_model, evidence_model):
  claim = Dropout(0.2)(claim_model.output)
  evidence = Dropout(0.2)(evidence_model.output)
  combined = Average()([claim, evidence])

  combined = Dense(64, activation='relu')(combined)
  combined = Dense(32, activation='relu')(combined)
  z = Dense(1, activation='sigmoid')(combined)
  return z

In [29]:
# encode class values as integers
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

encoder = LabelEncoder()
encoder.fit(y_train)

encoded_Y_train = encoder.transform(y_train)
encoded_Y_val = encoder.transform(y_val)
encoded_Y_test = encoder.transform(y_test)

In [30]:
model = {0:'LAST STATE', 1:'AVG STATES', 2:'MLP', 3:'BoW'}
multi = {0:'CONCATENATE', 1:'SUM', 2:'MEAN'}
for x in range(4):
  for y in range(3):
    if x == 0:
      print('LAST STATE')
      claim_outputs_fin, evidence_outputs_fin = last_state_model()
    elif x == 1:
      print('AVG STATES')
      claim_outputs_fin, evidence_outputs_fin = avg_model()
    elif x == 2:
      print('MLP')
      claim_outputs_fin, evidence_outputs_fin = mlp_model()
    elif x == 3:
      print('BoW')
      claim_outputs_fin, evidence_outputs_fin = bow_model()

    claim_model = Model(inputs=claim_input, outputs=claim_outputs_fin)
    evidence_model = Model(inputs=evidence_input, outputs=evidence_outputs_fin)

    sentence_similarity = Dot(axes=1)([claim_model.output, evidence_model.output]) 
    if y == 0:
      print('CONCATENATE')
      z = concat_model(claim_model, evidence_model)
    elif y == 1:
      print('SUM')
      z = sum_model(claim_model, evidence_model)
    elif y == 2:
      print('MEAN')
      z = mean_model(claim_model, evidence_model)

    model = Model(inputs=[claim_model.input, evidence_model.input], outputs=z)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit([train_sentences_X_claim, train_sentences_X_evidence], encoded_Y_train, batch_size=256, epochs=10, validation_data=([val_sentences_X_claim, val_sentences_X_evidence], encoded_Y_val))
    model.evaluate([val_sentences_X_claim, val_sentences_X_evidence], encoded_Y_val)
    y_pred = model.predict([test_sentences_X_claim, test_sentences_X_evidence])
    print(classification_report(encoded_Y_test, np.round(y_pred)))

LAST STATE
CONCATENATE
Epoch 1/10
476/476 [==============================] - 72s 152ms/step - loss: 0.5019 - accuracy: 0.7800 - val_loss: 0.6799 - val_accuracy: 0.6080
Epoch 2/10
476/476 [==============================] - 67s 141ms/step - loss: 0.4435 - accuracy: 0.8078 - val_loss: 0.6525 - val_accuracy: 0.6473
Epoch 3/10
476/476 [==============================] - 67s 141ms/step - loss: 0.4087 - accuracy: 0.8234 - val_loss: 0.6164 - val_accuracy: 0.6784
Epoch 4/10
476/476 [==============================] - 67s 141ms/step - loss: 0.3810 - accuracy: 0.8338 - val_loss: 0.6293 - val_accuracy: 0.6645
Epoch 5/10
476/476 [==============================] - 67s 141ms/step - loss: 0.3515 - accuracy: 0.8443 - val_loss: 0.6327 - val_accuracy: 0.6832
Epoch 6/10
476/476 [==============================] - 67s 141ms/step - loss: 0.3177 - accuracy: 0.8573 - val_loss: 0.6357 - val_accuracy: 0.6943
Epoch 7/10
476/476 [==============================] - 67s 140ms/step - loss: 0.2803 - accuracy: 0.8730 - va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3583
           1       0.50      1.00      0.67      3606

    accuracy                           0.50      7189
   macro avg       0.25      0.50      0.33      7189
weighted avg       0.25      0.50      0.34      7189

BoW
SUM
Epoch 1/10
476/476 [==============================] - 10s 20ms/step - loss: 0.5851 - accuracy: 0.7312 - val_loss: 0.8314 - val_accuracy: 0.5040
Epoch 2/10
476/476 [==============================] - 10s 20ms/step - loss: 0.5806 - accuracy: 0.7343 - val_loss: 0.8179 - val_accuracy: 0.5040
Epoch 3/10
476/476 [==============================] - 9s 20ms/step - loss: 0.5799 - accuracy: 0.7343 - val_loss: 0.7991 - val_accuracy: 0.5040
Epoch 4/10
476/476 [==============================] - 9s 20ms/step - loss: 0.5798 - accuracy: 0.7343 - val_loss: 0.8115 - val_accuracy: 0.5040
Epoch 5/10
476/476 [==============================] - 9s 20ms/step - loss: 0.5799 - accurac

# TRAINING

# EVALUATION

# COMMENTARY